In [4]:
import re
def split_text(text):
    
    text = text.replace('&gt;','>')
    split = text.split(' ')
    new_split = []

    for i in split:
        if i != '<a' and i != '' and 'href' not in i:
            new_split.append(i)
        
        if 'href' in i :
            start = i.find('>')
            end = i.find('<')

            if i[start+1:end] == 'PHTX':
                new_split.append(i[start+1:end])
                break

            new_split.append(i[start+1:end])

    return new_split


In [1]:
import requests, numpy, pandas, bs4, datetime

# step 1

request = requests.get('https://www.spaceweatherlive.com/en/solar-activity/top-50-solar-flares')

bs_object = bs4.BeautifulSoup(request.text,features='lxml')

content = str(bs_object.find('table'))

data_content = pandas.read_html(content)[0]

data_content.columns  = [ 'rank', 'x_classification', 'date', 'region','start_time', 'max_time', 'end_time', 'movie']

print(data_content)

    rank x_classification        date  region start_time max_time end_time  \
0      1             X28+  2003/11/04     486      19:29    19:53    20:06   
1      2             X20+  2001/04/02    9393      21:32    21:51    22:03   
2      3           X17.2+  2003/10/28     486      09:51    11:10    11:24   
3      4             X17+  2005/09/07     808      17:17    17:40    18:03   
4      5            X14.4  2001/04/15    9415      13:19    13:50    13:55   
5      6              X10  2003/10/29     486      20:37    20:49    21:01   
6      7             X9.4  1997/11/06    8100      11:49    11:55    12:01   
7      8             X9.3  2017/09/06    2673      11:53    12:02    12:10   
8      9               X9  2006/12/05     930      10:18    10:35    10:45   
9     10             X8.3  2003/11/02     486      17:03    17:25    17:39   
10    11             X8.2  2017/09/10    2673      15:35    16:06    16:31   
11    12             X7.1  2005/01/20     720      06:36    07:0

In [2]:
# step 2

data_content = data_content.iloc[:, :-1]

for index, row in data_content.iterrows():
    
    date = data_content['date'][index].split('/')
    
    start_time = data_content['start_time'][index].split(':')
    max_time = data_content['max_time'][index].split(':')
    end_time = data_content['end_time'][index].split(':')

    new_start_time = datetime.time(hour=int(start_time[0]), minute=int(start_time[1]))
    new_max_time = datetime.time(hour=int(max_time[0]), minute=int(max_time[1]))
    new_end_time = datetime.time(hour=int(end_time[0]), minute=int(end_time[1]))

    new_date = datetime.date(year=int(date[0]),month=int(date[1]),day=int(date[2]))

    data_content['start_time'][index] = datetime.datetime.combine(new_date, new_start_time)
    data_content['max_time'][index] = datetime.datetime.combine(new_date, new_max_time)
    data_content['end_time'][index] = datetime.datetime.combine(new_date, new_end_time)    

del data_content['date']
print(data_content)

    rank x_classification  region           start_time             max_time  \
0      1             X28+     486  2003-11-04 19:29:00  2003-11-04 19:53:00   
1      2             X20+    9393  2001-04-02 21:32:00  2001-04-02 21:51:00   
2      3           X17.2+     486  2003-10-28 09:51:00  2003-10-28 11:10:00   
3      4             X17+     808  2005-09-07 17:17:00  2005-09-07 17:40:00   
4      5            X14.4    9415  2001-04-15 13:19:00  2001-04-15 13:50:00   
5      6              X10     486  2003-10-29 20:37:00  2003-10-29 20:49:00   
6      7             X9.4    8100  1997-11-06 11:49:00  1997-11-06 11:55:00   
7      8             X9.3    2673  2017-09-06 11:53:00  2017-09-06 12:02:00   
8      9               X9     930  2006-12-05 10:18:00  2006-12-05 10:35:00   
9     10             X8.3     486  2003-11-02 17:03:00  2003-11-02 17:25:00   
10    11             X8.2    2673  2017-09-10 15:35:00  2017-09-10 16:06:00   
11    12             X7.1     720  2005-01-20 06:36:

/home/ahmedsamir/Desktop/data_science_project/env/lib/python3.6/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ahmedsamir/Desktop/data_science_project/env/lib/python3.6/site-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/ahmedsamir/Desktop/data_science_project/env/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [5]:
# step 3 

request = requests.get('https://cdaw.gsfc.nasa.gov/CME_list/radio/waves_type2.html')

bs_object = bs4.BeautifulSoup(request.text,features='lxml')

content = str(bs_object.find('pre'))

data_split_list = content.split('\n')

content_data = []

for i in range(12,len(data_split_list)-2):
    content_data.append(split_text(data_split_list[i]))

nasa_data_frame = pandas.DataFrame(content_data)

nasa_data_frame.columns = ['start_date','start_time','end_date','end_time','start_frequency',
                            'end_frequency','flare_location','flare_region','importance',
                            'cme_date','cme_time','cpa','width','spd','plots']
print(nasa_data_frame)

     start_date start_time end_date end_time start_frequency end_frequency  \
0    1997/04/01      14:00    04/01    14:15            8000          4000   
1    1997/04/07      14:30    04/07    17:30           11000          1000   
2    1997/05/12      05:15    05/14    16:00           12000            80   
3    1997/05/21      20:20    05/21    22:00            5000           500   
4    1997/09/23      21:53    09/23    22:16            6000          2000   
..          ...        ...      ...      ...             ...           ...   
513  2017/09/04      20:27    09/05    04:54           14000           210   
514  2017/09/06      12:05    09/07    08:00           16000            70   
515  2017/09/10      16:02    09/11    06:50           16000           150   
516  2017/09/12      07:38    09/12    07:43           16000         13000   
517  2017/09/17      11:45    09/17    12:35           16000           900   

    flare_location flare_region importance cme_date cme_time   

In [5]:
# step 4

nasa_data_frame['start_frequency'] = nasa_data_frame['start_frequency'].replace(r'[?]+', "NaN", regex=True)
nasa_data_frame['end_frequency'] = nasa_data_frame['end_frequency'].replace(r'[?]+', "NaN", regex=True)
nasa_data_frame['flare_location'] = nasa_data_frame['flare_location'].replace(r'[-]+', "NaN", regex=True)

nasa_data_frame['flare_region'] = nasa_data_frame['flare_region'].replace(r'[a-zA-Z]+', "NaN", regex=True)
nasa_data_frame['flare_region'] = nasa_data_frame['flare_region'].replace(r'[-]+', "NaN", regex=True)

nasa_data_frame['importance'] = nasa_data_frame['importance'].replace(r'[-]+', "NaN", regex=True)

nasa_data_frame['cme_date'] = nasa_data_frame['cme_date'].replace(r'--/--', "NaN", regex=True)

nasa_data_frame['cme_time'] = nasa_data_frame['cme_time'].replace(r'--:--', "NaN", regex=True)

nasa_data_frame['cpa'] = nasa_data_frame['cpa'].replace(r'[-]+', "NaN", regex=True)
nasa_data_frame['width'] = nasa_data_frame['width'].replace(r'[-]+', "NaN", regex=True)
nasa_data_frame['spd'] = nasa_data_frame['spd'].replace(r'[-]+', "NaN", regex=True)

nasa_data_frame['width_lower_bound'] = [True if '>' in i else False for i in nasa_data_frame['width']]
nasa_data_frame['width'] = [i if '>' not in i else i[1:] for i in nasa_data_frame['width']]

start_date_list = [x.split('/') for x in nasa_data_frame['start_date']]
start_time_list = [x.split(':') for x in nasa_data_frame['start_time']]
start_date_time_list = [datetime.datetime.combine(
                datetime.date(year=int(start_date_list[x][0]),
                            month=int(start_date_list[x][1]),
                            day=int(start_date_list[x][2])),
                
                datetime.time(
                            hour=int(start_time_list[x][0]),
                            minute=int(start_time_list[x][1]))) 
                for x in range(len(start_date_list))]

nasa_data_frame['start_datetime'] = start_date_time_list

end_date_list = [x.split('/') for x in nasa_data_frame['end_date']]
end_time_list = [x.split(':') for x in nasa_data_frame['end_time']]

end_date_time_list = [datetime.datetime.combine(
                datetime.date(year=int(start_date_list[x][0]),
                            month=int(end_date_list[x][0]),
                            day=int(end_date_list[x][1])),
                
                datetime.time(
                            hour=int(end_time_list[x][0]) if end_time_list[x][0] != '24' else 0,
                            minute=int(end_time_list[x][1]))) 
                for x in range(len(end_time_list))]

nasa_data_frame['end_datetime'] = end_date_time_list

cme_date_list = [x.split('/') for x in nasa_data_frame['cme_date']]
cme_time_list = [x.split(':') for x in nasa_data_frame['cme_time']]

cme_date_time_list = []

for x in range(len(cme_date_list)):

    if 'NaN' not in cme_date_list[x] and x != len(cme_date_list)-1:
        cme_date_time_list.append(
                            datetime.datetime.combine(
                            datetime.date(year=int(start_date_list[x][0]),
                            month=int(cme_date_list[x][0]),
                            day=int(cme_date_list[x][1])),
                
                            datetime.time(
                            hour=int(cme_time_list[x][0]),
                            minute=int(cme_time_list[x][1])))
                        )
    else:
        cme_date_time_list.append('NaN')


nasa_data_frame['cme_datetime'] = cme_date_time_list

del nasa_data_frame['start_date']
del nasa_data_frame['end_date']
del nasa_data_frame['start_time']
del nasa_data_frame['end_time']
del nasa_data_frame['cme_date']
del nasa_data_frame['cme_time']

nasa_data_frame['is_halo'] = [True if x == 'Halo' else False for x in nasa_data_frame['cpa']]
nasa_data_frame['cpa'] = [x if x != 'Halo' else 'NaN' for x in nasa_data_frame['cpa']]

In [6]:
print(data_content)

    rank x_classification  region           start_time             max_time  \
0      1             X28+     486  2003-11-04 19:29:00  2003-11-04 19:53:00   
1      2             X20+    9393  2001-04-02 21:32:00  2001-04-02 21:51:00   
2      3           X17.2+     486  2003-10-28 09:51:00  2003-10-28 11:10:00   
3      4             X17+     808  2005-09-07 17:17:00  2005-09-07 17:40:00   
4      5            X14.4    9415  2001-04-15 13:19:00  2001-04-15 13:50:00   
5      6              X10     486  2003-10-29 20:37:00  2003-10-29 20:49:00   
6      7             X9.4    8100  1997-11-06 11:49:00  1997-11-06 11:55:00   
7      8             X9.3    2673  2017-09-06 11:53:00  2017-09-06 12:02:00   
8      9               X9     930  2006-12-05 10:18:00  2006-12-05 10:35:00   
9     10             X8.3     486  2003-11-02 17:03:00  2003-11-02 17:25:00   
10    11             X8.2    2673  2017-09-10 15:35:00  2017-09-10 16:06:00   
11    12             X7.1     720  2005-01-20 06:36:

In [7]:
print(nasa_data_frame)

    start_frequency end_frequency flare_location flare_region importance  cpa  \
0              8000          4000         S25E16         8026       M1.3   74   
1             11000          1000         S28E19         8027       C6.8  NaN   
2             12000            80         N21W08         8038       C1.3  NaN   
3              5000           500         N05W12         8040       M1.3  263   
4              6000          2000         S29E25         8088       C1.4  133   
..              ...           ...            ...          ...        ...  ...   
379           16000           200           BACK          NaN        NaN  NaN   
380           16000           300           BACK          NaN        NaN  NaN   
381           16000          1500           BACK          NaN        NaN  NaN   
382            4000          1500           BACK          NaN        NaN  261   
383           16000          5000         N20W65        11451       C3.9  NaN   

    width   spd plots  widt